In [1]:
x=1

In [13]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import multigrate as mtg
import multimil as mtm
import scanpy as sc
import ast
import anndata as ad
import yaml
from pprint import pprint

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [37]:
df1 = pd.read_csv('../data/multigrate/hlca_small/8c0502466b/0/checkpoints/epoch=49-step=24450.csv')
df1['split'] = 1
df1['method'] = 'test'
df1

,Unnamed: 0,precision,recall,f1-score,support,split,method
0,COVID-19,0.000000,0.000000,0.000000,4636.000000,1,test
1,Healthy,0.866640,0.981527,0.920513,30694.000000,1,test
2,accuracy,0.852731,0.852731,0.852731,0.852731,1,test
3,macro avg,0.433320,0.490764,0.460256,35330.000000,1,test
4,weighted avg,0.752919,0.852731,0.799723,35330.000000,1,test


In [38]:
df2 = {}
df2['split'] = 2
df2['method'] = 'multigrate'
df2 = pd.DataFrame.from_dict(df2, orient='index').T
df2

,split,method
0,2,multigrate


In [39]:
dfs = [df1, df2]
df = pd.concat(dfs)
df

,Unnamed: 0,precision,recall,f1-score,support,split,method
0,COVID-19,0.000000,0.000000,0.000000,4636.000000,1,test
1,Healthy,0.866640,0.981527,0.920513,30694.000000,1,test
2,accuracy,0.852731,0.852731,0.852731,0.852731,1,test
3,macro avg,0.433320,0.490764,0.460256,35330.000000,1,test
4,weighted avg,0.752919,0.852731,0.799723,35330.000000,1,test
0,NaN,NaN,NaN,NaN,NaN,2,multigrate


In [25]:
method = 'multigrate_mil'
task = 'hlca_healthy_ipf_embed_balanced'
df_best = df[(df['task'] == task) & (df['method'] == method)]
df_best

,hash,accuracy,best_params,best_epoch,best_query_epoch,accuracies,method,task
2,b2e8c62cf5,0.992275,"{'lr': 0.001, 'rna_indices_end': 30, 'categori...",39.0,NaN,"[0.9864115021332216, 0.9981378507694296, 0.828...",multigrate_mil,hlca_healthy_ipf_embed_balanced


In [26]:
assert(len(df_best) == 1)